In [1]:
from utility.pcd2img import pcd2img_np
from trained_model.yolo_detect import Detect
from utility.get_coords import scale_pred_to_xy_point_cloud, draw_coord_on_img, scale_coord, get_strides
from utility.generate_tree import get_tree_many_from_coords, get_h_from_each_tree_slice
from utility.csf_py import csf_py

# Standard Libraries
import os
import cv2
import yaml
import open3d as o3d
import pandas as pd
import numpy as np
import time

start = time.time()
with open("config/config.yaml", 'r') as ymlfile:
    yml_data = yaml.load(ymlfile, Loader = yaml.FullLoader)

# Input Folder Location
curr_dir = os.getcwd()
folder_loc = yml_data["dataset"]["folder_location"]
pcd_filename = yml_data["dataset"]["pcd"]["file_name"] + yml_data["dataset"]["pcd"]["file_type"]

# Output Folder Location
output_folder = os.getcwd() + yml_data["output"]["folder_location"] +"/"+ yml_data["dataset"]["pcd"]["file_name"] +"/"
topViewOut = output_folder + yml_data["output"]["topView"]["folder_location"]
sideViewOut = output_folder + yml_data["output"]["sideView"]["folder_location"]
csvOut = output_folder + yml_data["dataset"]["pcd"]["file_name"] +".csv"
pcd_name = yml_data["dataset"]["pcd"]["file_name"]

# Read pcd
print("Reading pcd file...")
pcd = o3d.io.read_point_cloud(folder_loc+pcd_filename)

assert len(pcd.points) >= 1, f"Failed to Read PCD [{pcd_filename}], it's Empty"

print("Reading PCD file successful, Generating stuff")
for path in [output_folder, topViewOut, sideViewOut]:
    if not os.path.exists(path):
        os.mkdir(path)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Reading pcd file...
Reading PCD file successful, Generating stuff


### 1. CSF and Rasterize Top View
1. Load Yolov5 Model
2. CSF and Rasterize to image Based on set Params
    - Relief : SloopSmooth = True, res = Medium(7.0), Threshold = Med(1.0), rigidness = 3
    - Steep Slope: SloopSmooth=True, res = High(15.0), Threshold = High(2.0), rigidness = 1


In [2]:
# Yaml Params
topViewStepsize = yml_data["yolov5"]["topView"]["stepsize"]
top_view_model_pth = yml_data["yolov5"]["topView"]["model_pth"]
yolov5_folder_pth = yml_data["yolov5"]["yolov5_pth"]
ideal_img_size = yml_data["yolov5"]["topView"]["imgSize"]

# 1. Generate Top View Yolov5 Model
topViewModel = Detect(yolov5_folder_pth, top_view_model_pth, img_size=ideal_img_size)

grd, non_grd = csf_py(
    pcd, 
    return_non_ground = "both", 
    bsloopSmooth = True, 
    cloth_res = 15.0, 
    threshold= 2.0, 
    rigidness=1
)
# 2. Create img from CSF
non_ground_img = pcd2img_np(non_grd,"z",topViewStepsize)

INFO - 2022-05-12 13:10:17,110 - torch_utils - YOLOv5 🚀 v6.1-82-g71621df torch 1.11.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1070, 8105MiB)

INFO - 2022-05-12 13:10:21,020 - yolo - Fusing layers... 
INFO - 2022-05-12 13:10:21,239 - torch_utils - Model summary: 290 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
INFO - 2022-05-12 13:10:21,243 - common - Adding AutoShape... 


img_size:  [640, 640]
Device :  cuda
[0] Configuring terrain...
[0] Configuring cloth...
[0]  - width: 19 height: 19
[0] Rasterizing...
[0] Simulating...
[0]  - post handle...


### 2. Split Large Image into Multiple Images
The reason for this step is to make sure the Images are within the scale of Yolov5 Top View Model. To get the ideal and maximum effectiveness for Tree Height detection
1. calculate spacing, then use linspace for even spacing
2. Split the Images
    - For each Split image, run inference with yolov5
    - Merge and scale the Coordinates of each image into one list
4. Scale 2D coordinates to 3D

In [3]:
# Get Coordinates from Top View
coordinates = []

# 1. Calculate spacing for image splitting
h_s, w_s = get_strides(non_ground_img.shape, ideal_img_size)
h_arr, h_incre = np.linspace(0, non_ground_img.shape[0], h_s+1, retstep=True)
w_arr, w_incre = np.linspace(0, non_ground_img.shape[1], w_s+1, retstep=True)

# 2. Split images 
for i, h in enumerate(h_arr[:-1]):
    for j, w in enumerate(w_arr[:-1]):
        img = non_ground_img[int(round(h)):int(round(h+h_incre)), int(round(w)):int(round(w+w_incre))]
        preds = topViewModel.predict(
            img,
            convert_to_gray=False,
            confi_thres = 0.134,
            iou_thres = 0.02
            )
        coordinates.extend(scale_pred_to_xy_point_cloud(preds, 1, w, h))
        del img, preds

# 2c Visualization Purpose
img_with_coord = draw_coord_on_img(non_ground_img, np.asarray(coordinates), circle_size=10)
cv2.imwrite(f"{topViewOut}/{pcd_name}_coor.png", img_with_coord)

# 3. Scale 2D to 3D
xmin, ymin, zmin = non_grd.get_min_bound()
xmax, ymax, zmax = non_grd.get_max_bound()
range_x, range_y, range_z = xmax-xmin, ymax-ymin, zmax-zmin

height, width = non_ground_img.shape
coordinates = scale_coord(
    np.asarray(coordinates), 
    scale=(range_x/width, range_y/height), 
    offset=(xmin,-ymax)
    )

# 4. Clear unused memory
del topViewModel
del non_grd

### 3. Generate Height Map from Coordinates (x,y)
1. Initialize Yolov5 Side View Model
2. Crop Each Tree From specified Coordinates
    - Slice the Point cloud in X and Y axis
        - Run Yolov5 Inference on each Slice to find Height
        - Store Height on Each Slice
    - Current Algo: Get the maximum Height of all slice
    - Store [x,y and height] in coords_hs list

In [4]:
# Yaml Params
side_view_model_pth = yml_data["yolov5"]["sideView"]["model_pth"]
side_view_step_size = yml_data["yolov5"]["sideView"]["stepsize"]
side_view_img_size = tuple(yml_data["yolov5"]["sideView"]["imgSize"])
min_points_per_tree = yml_data["yolov5"]["sideView"]["minNoPoints"]

# Init SideViewYolo Model
sideViewModel = Detect(yolov5_folder_pth, side_view_model_pth, img_size=side_view_img_size)
coords_hs = []
ex_w, ex_h = (dim*side_view_step_size for dim in side_view_img_size)
values = get_tree_many_from_coords(pcd, grd, coordinates,expand=[ex_w,ex_w,ex_h])

# Visualization Purposes
# trees = values[:,0].tolist()
# bBoxes = values[:,1].tolist()
# o3d.visualization.draw_geometries(bBoxes+trees+[pcd])

INFO - 2022-05-12 13:11:02,963 - torch_utils - YOLOv5 🚀 v6.1-82-g71621df torch 1.11.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1070, 8105MiB)

INFO - 2022-05-12 13:11:04,493 - yolo - Fusing layers... 
INFO - 2022-05-12 13:11:04,707 - torch_utils - Model summary: 290 layers, 20856975 parameters, 0 gradients, 48.0 GFLOPs
INFO - 2022-05-12 13:11:04,711 - common - Adding AutoShape... 


img_size:  [320, 768]
Device :  cuda


In [5]:
# Generate Height from each Tree
# 1. Filter out Trees that are below a threshold of points
# 2. Generate Image for each point cloud
# 3. Calculate Height of Each tree from Image
# 4. Scale the value
# 5. Clear memory
print(len(values))

for index, value in enumerate(values):
    tree, _, x,y = value
    h = get_h_from_each_tree_slice(
        tree = tree,
        model = sideViewModel,
        img_size = side_view_img_size, 
        stepsize = side_view_step_size,
        img_dir = f"{sideViewOut}/{pcd_name}_{index}_",
        gen_img = True,
        img_with_h = False,
        min_no_points = min_points_per_tree
        )
    coords_hs.append((x,y,h,index)) if h > 0 else None


del sideViewModel, pcd, grd

358


### 4. Export Heights to CSV

In [6]:
# Yaml Params
df = pd.DataFrame(coords_hs, columns=["x","y","h","index"])
df.to_csv(csvOut)

# Sent

### 5. Calculate total time taken

In [7]:
end = time.time()
print("Total Time Taken: ", end - start)

Total Time Taken:  126.55844140052795


#### 6. Draw height on image

In [8]:
def draw_height_on_coord(img, x_y_h:np.ndarray, on_left:str="tape"): #(xc,yc,confidence,label)
    img = img.copy()
    if not x_y_h.size:
        return img
    red = (255,0,0)
    color = red
    font = cv2.FONT_HERSHEY_SIMPLEX
    for stuff in x_y_h:
        x,y,h,index = int(stuff[0]), int(stuff[1]), stuff[2], stuff[3]
        img = cv2.circle(img,(x,y), 1, color,5)
        img = cv2.putText(img, f"{h:.1f}", (x,y),font, 0.7,red,2,cv2.LINE_AA)
        img = cv2.putText(img, f"{int(index)}", (x-16,y-20),font, 0.7,red,2,cv2.LINE_AA)
    # Writing image name
    img = cv2.putText(img, on_left, (50,50), font, 2, red, 2, cv2.LINE_AA)
    return img

new_pd = np.asarray([df.x, df.y, df.h, df.index]).transpose()
new_pd = scale_coord(new_pd, (1/topViewStepsize, 1/topViewStepsize), (-xmin/topViewStepsize,ymax/topViewStepsize))
img_lidar = draw_height_on_coord(non_ground_img, new_pd, f"Lidar {pcd_name}")


cv2.imwrite(f"{topViewOut}/{pcd_name}_lidar.png", img_lidar)

True